In [ ]:
%pip install openai

In [ ]:
# %%writefile -a ./.env

# AZURE_OPENAI_API_KEY=
# AZURE_OPENAI_ENDPOINT=

In [ ]:
import os
from openai import AzureOpenAI
import base64

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Function to encode image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "OpenAI_demo_files/the_last_supper.jpeg"
base64_image = encode_image(image_path)

# Create the message with both text and image
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "What can you see in this image? If there are any people, what are they doing?"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                }
            }
        ]
    }
]

# Send request to GPT-4V
response = client.chat.completions.create(
    model="gpt-4o", 
    messages=messages,
    max_tokens=300
)

# Print the response
print(response.choices[0].message.content)

The image is a depiction of "The Last Supper," a famous painting by Leonardo da Vinci. The painting shows a large table with a white tablecloth, with a variety of dishes, cups, and food items placed on it. There are several people seated at the table, and they appear to be engaged in conversation and expressive gestures. The interior setting includes a doorway and windows with a view of a landscape outside. The overall composition is a well-known artistic representation of the biblical scene where Jesus dines with his disciples.


In [ ]:
from openai import AzureOpenAI
import os
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-05-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

assistant = client.beta.assistants.create(
  name="Financial Analyst Assistant",
  instructions="You are an expert financial analyst. Use your knowledge base to answer questions about audited financial statements.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

# Create a vector store called "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Financial Statements")
 
# Ready the files for upload to OpenAI
file_paths = ["OpenAI_demo_files/financial_results.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [8]:
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are the major source of revenue for the company?"
)

# Run the thread and poll for the result
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please answer based on files provided",
)

if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for message in messages.data[::-1]:
        print(f"{message.role}: {'/n'.join([x.text.value for x in message.content])}")

user: What are the major source of revenue for the company?
assistant: The major sources of revenue for the company are:

1. **Subscription Revenue**: This is the largest source of revenue for the company, amounting to $5.06 billion in the most recent quarter. It includes revenue from the company's main product lines: Creative Cloud, Document Cloud, and Experience Cloud.
2. **Product Revenue**: This includes revenue from the sale of products and amounted to $104 million in the most recent quarter.
3. **Services and Other Revenue**: This is the revenue generated from professional services, support and training, and other miscellaneous sources. It amounted to $145 million in the most recent quarter【4:3†source】【4:5†source】.


In [26]:
message.content[0].text.annotations

[FileCitationAnnotation(end_index=229, file_citation=FileCitation(file_id='assistant-JqjPU1A0JQtarNcnzFwAbNc5'), start_index=217, text='【4:1†source】', type='file_citation'),
 FileCitationAnnotation(end_index=229, file_citation=FileCitation(file_id='assistant-JqjPU1A0JQtarNcnzFwAbNc5'), start_index=217, text='【4:1†source】', type='file_citation'),
 FileCitationAnnotation(end_index=328, file_citation=FileCitation(file_id='assistant-JqjPU1A0JQtarNcnzFwAbNc5'), start_index=316, text='【4:1†source】', type='file_citation'),
 FileCitationAnnotation(end_index=328, file_citation=FileCitation(file_id='assistant-JqjPU1A0JQtarNcnzFwAbNc5'), start_index=316, text='【4:1†source】', type='file_citation'),
 FileCitationAnnotation(end_index=416, file_citation=FileCitation(file_id='assistant-JqjPU1A0JQtarNcnzFwAbNc5'), start_index=404, text='【4:1†source】', type='file_citation'),
 FileCitationAnnotation(end_index=416, file_citation=FileCitation(file_id='assistant-JqjPU1A0JQtarNcnzFwAbNc5'), start_index=404, 

In [ ]:

# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are the major source of revenue for the company?"
)

# Run the thread and poll for the result
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please answer based on files provided",
)

if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for message in messages.data[::-1]:
        print(f"{message.role}: {'/n'.join([x.text.value for x in message.content])}")